# Exploring Ebay Car Sales Data

In this project we will work with a dataset of used cars from eBay Kleinanzeigen, a [classifieds](https://en.wikipedia.org/wiki/Classified_advertising) section of the German eBay website.

The dataset was initially scraped and uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). However, [Dataquest](https://www.dataquest.io) modified the original dataset from Kaggle by:

* Sampling 50,000 data points from the full dataset, to reduce memory usage and ensure a faster runtime (which is ideal for this exploratory project)


* They also dirtied the dataset a bit to more closely resemble what would be expected from a scraped dataset (as the version uploaded to Kaggle was already cleaned to be easier to work with).

## Introduction

The data dictionary provided with the data is as follows:

* dateCrawled - When this ad was first crawled. All field-values are taken from this date.


* name - Name of the car.


* seller - Whether the seller is private or a dealer.


* offerType - The type of listing


* price - The price on the ad to sell the car.


* abtest - Whether the listing is included in an A/B test.


* vehicleType - The vehicle Type.


* yearOfRegistration - The year in which the car was first registered.


* gearbox - The transmission type.


* powerPS - The power of the car in PS.


* model - The car model name.


* odometer - How many kilometers the car has driven.


* monthOfRegistration - The month in which the car was first registered.


* fuelType - What type of fuel the car uses.


* brand - The brand of the car.


* notRepairedDamage - If the car has a damage which is not yet repaired.


* dateCreated - The date on which the eBay listing was created.


* nrOfPictures - The number of pictures in the ad.


* postalCode - The postal code for the location of the vehicle.


* lastSeenOnline - When the crawler saw this ad last online.


The objective of this project is to clean, then analyze the included used car listings. Lets start by importing the libraries needed for the data cleaning and analysis process, then reading in the dataset using pandas.

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv',encoding='Latin-1')

Next we will get some information on the dataset and display its first few rows using the df.info() and df.head() methods.

In [2]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

As can be seen from the resulting information on the autos dataframe, there are 50,000 rows and 20 columns. Of the 20 columns, most of them are strings. It can also be seen that the memory usage of this dataset is about 7.6 MB and there are a few columns such as vehicleType and fuelType which have some null values.

## Cleaning Column Names

An important observation regarding the data is that the column names use [camelcase](https://en.wikipedia.org/wiki/Camel_case) instead of Python's preferred [snakecase](https://en.wikipedia.org/wiki/Snake_case), which means that spaces can't replaced with underscores.

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive. First we'll print an array of the existing column names.

In [4]:
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


Now we make the changes below then assign the modified columns names back to the autos dataframe.

In [5]:
columns = list(autos.columns)
columns[0] = 'crawled_date'
columns[3] = 'offer_type'
columns[5] = 'ab_test'
columns[6] = 'vehicle_type'
columns[7] = 'registration_year'
columns[8] = 'gear_box'
columns[9] = 'power_ps'
columns[12] = 'registration_month'
columns[13] = 'fuel_type'
columns[15] = 'unrepaired_damage'
columns[16] = 'ad_created'
columns[17] = 'nr_of_pictures'
columns[18] = 'postal_code'
columns[19] = 'last_seen'
autos.columns = columns

Next, lets examine the current state of the autos dataframe.

In [6]:
autos.head()

,crawled_date,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Initial Exploration and Cleaning

We will now do some simple data exploration to find out what other data cleaning tasks are necessary. To begin with, we will try to identify:

* Text columns where all or almost all of the values are the same. These can usually be discarded as they don't contain useful information for analysis.

* Examples of numeric data stored as text which can be cleaned and converted.

First let us look at descriptive statistics for all columns using the df.describe() method.

In [7]:
# include='all' is used to get both categorical
# and numerical columns.
autos.describe(include='all')

,crawled_date,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


It can be observed from above that a few text columns such as seller, offer_type and ab_test only have 2 unique values. It will be worth investigating such columns to see how these unique values are distributed. If a column is predominantly made up of just one unique value, then this column may be dropped as stated in the criteria above. The series.value_counts() method will be used to have a closer look at the columns with only a few unique values.

In [8]:
autos['seller'].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

In [9]:
autos['offer_type'].value_counts()

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64

In [10]:
autos['ab_test'].value_counts()

test       25756
control    24244
Name: ab_test, dtype: int64

In [11]:
autos['vehicle_type'].value_counts()

limousine     12859
kleinwagen    10822
kombi          9127
bus            4093
cabrio         3061
coupe          2537
suv            1986
andere          420
Name: vehicle_type, dtype: int64

In [12]:
autos['gear_box'].value_counts()

manuell      36993
automatik    10327
Name: gear_box, dtype: int64

In [13]:
autos['fuel_type'].value_counts()

benzin     30107
diesel     14567
lpg          691
cng           75
hybrid        37
andere        22
elektro       19
Name: fuel_type, dtype: int64

In [14]:
autos['unrepaired_damage'].value_counts()

nein    35232
ja       4939
Name: unrepaired_damage, dtype: int64

From the analysis above 2 columns emerged as candidates to be dropped. In the seller and offer_type columns, 99.9% of the values were the same. For the seller column, all but one of the 50,000 sellers were private (privat) and for the offer_type column, all but one of the 50,000 offer types were offers (Angebot). These columns can therefore be discarded due to a lack of helpful information for further analysis. These columns will now be dropped using the df.drop() method.

In [15]:
autos.drop(columns = ['seller','offer_type'],inplace=True)

A quick check to confirm that these columns have now been removed.

In [16]:
autos.head()

,crawled_date,name,price,ab_test,vehicle_type,registration_year,gear_box,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Next, it can be discerned that the price and odometer columns contain numeric values stored as text. So for each column we will:

* Remove any non-numeric characters and

* Convert the column to a numeric dtype.

* Then rename the column so as not to lose information.

In [17]:
autos['price'] = autos['price'].str.replace('$','').str.replace(',','').astype(int)
autos['odometer'] = autos['odometer'].str.replace(',','').str.replace('km','').astype(int)
autos.rename(columns={'price':'price_dollars','odometer':'odometer_km'},inplace=True)

A quick to check to confirm all the changes.

In [18]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 18 columns):
crawled_date          50000 non-null object
name                  50000 non-null object
price_dollars         50000 non-null int64
ab_test               50000 non-null object
vehicle_type          44905 non-null object
registration_year     50000 non-null int64
gear_box              47320 non-null object
power_ps              50000 non-null int64
model                 47242 non-null object
odometer_km           50000 non-null int64
registration_month    50000 non-null int64
fuel_type             45518 non-null object
brand                 50000 non-null object
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
nr_of_pictures        50000 non-null int64
postal_code           50000 non-null int64
last_seen             50000 non-null object
dtypes: int64(7), object(11)
memory usage: 6.9+ MB


## Exploring the Odometer and Price Columns

Let's continue exploring the data, specifically looking for data that doesn't look right. We'll begin by examining the odometer_km and price_dollars columns. Here are the steps we'll take:

* Examine the columns using minimum and maximum values and search for any values that appear unrealistically high or low (outliers) that we might want to remove.


* We'll use series.unique().shape to see how many unique values


* Series.describe() to view min/max/median/mean etc


* Series.value_counts() with series.sort_index() to see the highest and lowest values with their counts.

In [19]:
autos['odometer_km'].unique().shape

(13,)

In [20]:
autos['price_dollars'].unique().shape

(2357,)

So, odometer_km has 13 unique values, while price_dollars has 2357 unique values.

In [21]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

A glance at this description for the odometer_km column suggests that outliers may exist in the lower range of the data as the minimum value (5,000) is more than 2 standard deviations away from the mean and it is a lot less than the first quartile (25% mark) of the data.

However, it can be seen from the descriptive statistics above that the median (50% mark), third quartile (75% mark) and maximum values are all the same (150,000), which implies that a large distribution of the cars in the dataset have travelled 150,000 km. This will be investigated further.

In [22]:
autos['price_dollars'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_dollars, dtype: float64

A glance at this description for the price_dollars column suggests that there may be outliers in the lower range and upper range of the data as the minimum and maximum values are more than 2 standard deviations away from the mean. Also the maximum value is very far away from the third quartile  of the data. In addition, the minimum value of the data appears to be zero which is highly unlikely in real life as this would imply that a car was sold for free. This will be analyzed further.

The odometer_km column only has 13 unique values as was determined earlier so we can proceed to use the series.value_counts() method to view the distribution of the column. We will also apply the series.sort_index() method to sort out the column.

In [23]:
autos['odometer_km'].value_counts().sort_index()

5000        967
10000       264
20000       784
30000       789
40000       819
50000      1027
60000      1164
70000      1230
80000      1436
90000      1757
100000     2169
125000     5170
150000    32424
Name: odometer_km, dtype: int64

Upon further review of the odometer_km column, it becomes clear that the minimum value of 5,000 km travelled by a car is not actually an outlier. The reason why it appears to be an outlier on first glance of the descriptive statistics is because the maximum value of 150,000 km travelled by a car accounts for about 65% of all the values, which has the effect of skewing the mean towards the upper end of the data.

The price_dollars column has 2000+ unique values as was determined earlier, so instead of using the series.value_counts() method directy to the column, we will use it in combination with the series.sort_index() and series.head() methods to observe the lower and upper end of the column's distribution.

In [24]:
autos["price_dollars"].value_counts().sort_index().head(25)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
40       6
45       4
47       1
49       4
50      49
Name: price_dollars, dtype: int64

It appears that 1421 cars in the dataset cost $0. As this makes up only about 3% of all the cars, we may decide to discard this. Let's now look at the highest values.

In [25]:
autos['price_dollars'].value_counts().sort_index(ascending=False).head(25)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
194000      1
190000      1
180000      1
175000      1
169999      1
Name: price_dollars, dtype: int64

The highest price of a car here of about \\$100 million seems excessive and infact as can be seen above, the prices of cars rise steadily to about $350,000 before drastically increasing. Since the car sales above \\$350,000 only account for less than 0.005% of all the car sales, we can treat those prices as outliers and get rid of them.

As eBay effectively operates as a site for auctions, it is not uncommon for bidding to start at \\$1, so our updated price range will be from $1 to \$350,000. This is achieved below.

In [26]:
autos = autos[autos['price_dollars'].between(1,350000)]
autos['price_dollars'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price_dollars, dtype: float64

## Exploring the date columns

Let's now move on to the date columns and understand the date range the data covers. 

There are 5 columns that should represent date values. Some of these columns were created by the data crawler and some came from the website itself. We can differentiate by referring to the data dictionary:

* 'crawled_date': added by the crawler
* 'last_seen': added by the crawler
* 'ad_created': from the website
* 'registration_month': from the website
* 'registration_year': from the website

Right now the date_crawled, last_seen, and ad_created columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can interpret it quantitatively. The other two columns are represented as numeric values, so we can use methods like Series.describe() to understand the distribution without any extra data processing.

Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values, like so:

In [27]:
autos[['crawled_date','ad_created','last_seen']].head()

,crawled_date,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


You'll notice that the first 10 characters represent the day (e.g. 2016-03-12). To understand the date range, we can extract just the date values, use Series.value_counts() to generate a distribution, and then sort by the index.

To select the first 10 characters in each column, we can use Series.str[:10]:

In [28]:
print(autos['crawled_date'].str[:10].head())

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: crawled_date, dtype: object


Using the workflow described above, we will now calculate the distribution of values in the crawled_date, ad_created, and last_seen columns (all string columns) as percentages.

To include missing values in the distribution and to use
percentages instead of counts, chain the
Series.value_counts(normalize=True,dropna=False) method.

In [29]:
print(autos['crawled_date']
      .str[:10]
      .value_counts(normalize=True,dropna=False)
      .sort_index())

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: crawled_date, dtype: float64


It looks as if the data was crawled everyday from early March to early April 2016. The values seem to be evenly distributed.

In [30]:
print(autos['last_seen']
      .str[:10]
      .value_counts(normalize=True,dropna=False)
      .sort_index())

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64


The data crawler documented the date it last saw any cars listed online, which lets us deduce when a listing was removed, presumably because the car was sold.

The last few days comprise a disproportionate amount of 'last seen' values. Given that these are 6 to 10 times the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the data crawling period ending and don't imply car sales.

In [31]:
print(autos['ad_created']
      .str[:10]
      .value_counts(normalize=True,dropna=False)
      .sort_index())

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

There is a wide range of ad created dates. Most fall within a month or 2 months of the listing date, but a few of them are quite old, with the oldest at around 9 months. Now lets look at the registration_month and registration_year columns.

## Dealing with the Registration Month and Registration Year data

In [32]:
autos['registration_month'].describe()

count    48565.000000
mean         5.782251
std          3.685595
min          0.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64

It can be noticed that for registration_month there is a minimum value of 0 and a maximum value of 12, suggesting that 13 months were considered, when there are obviously only 12 months in a year. Presuming that the registration month column categorises 1 to 12 as January to December, then the value 0 should be an error. This will be investigated further below to determine if it's an outlier or not.

In [33]:
autos['registration_month'].value_counts().sort_index()

0     4480
1     3219
2     2937
3     5003
4     4036
5     4031
6     4271
7     3857
8     3126
9     3330
10    3588
11    3313
12    3374
Name: registration_month, dtype: int64

There are almost 4500 entries with a registration month of 0 (about 9% of all values). So clearly this is not just an outlier and may represent an error in the dataset. A possible explanation for this could be that when the data was scraped from the website, some of the fields did not have a registration month and a zero was put in by default for these.

Whatever the reason, those entries are now candidates to be dropped from the dataset but for now let's consider the registration_year column.

In [34]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The minimum and maximum values for the registration year of 1000 and 9999 immediately stand out as red flags. In the year 1000, cars had not been invented yet and we are still centuries away from 9999. Let's inspect the entries for these years.

In [35]:
autos[autos['registration_year']==1000]

,crawled_date,name,price_dollars,ab_test,vehicle_type,registration_year,gear_box,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
22316,2016-03-29 16:56:41,VW_Kaefer.__Zwei_zum_Preis_von_einem.,1500,control,NaN,1000,manuell,0,kaefer,5000,0,benzin,volkswagen,NaN,2016-03-29 00:00:00,0,48324,2016-03-31 10:15:28


In [36]:
autos[autos['registration_year']==9999]

,crawled_date,name,price_dollars,ab_test,vehicle_type,registration_year,gear_box,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
8012,2016-03-23 16:43:29,Opel_GT_Karosserie_mit_Brief!,700,test,NaN,9999,NaN,0,andere,10000,0,NaN,opel,NaN,2016-03-23 00:00:00,0,21769,2016-04-05 20:16:15
33950,2016-03-23 21:52:25,58er_karmann_ghia_lowlight_Kaefer__zum_restaur...,7999,test,NaN,9999,NaN,0,kaefer,10000,0,NaN,volkswagen,NaN,2016-03-23 00:00:00,0,47638,2016-04-06 03:46:40
38076,2016-04-04 22:54:47,Mercedes_Benz_A180,18000,test,NaN,9999,NaN,0,a_klasse,10000,0,benzin,mercedes_benz,NaN,2016-04-04 00:00:00,0,51379,2016-04-07 02:44:52


There is only one car entry with the registration year of 1000 and three with the registration year of 9999, so these entries can be easily removed from the dataset. Interestingly, the registration month for these 4 entries are all 0, so they were already candidates to be dropped from the dataset.

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [37]:
autos[(autos['registration_year']<1900)
      |(autos['registration_year']>2016)]['registration_year'].count()

1884

In [38]:
autos[(autos['registration_year']<1900)
      |(autos['registration_year']>2016)]['registration_year'].value_counts()

2017    1392
2018     470
5000       4
9999       3
2019       2
1800       2
2800       1
6200       1
4500       1
8888       1
4800       1
1000       1
9000       1
1001       1
1111       1
5911       1
4100       1
Name: registration_year, dtype: int64

So there are 1884 listings of cars that fall outside the 1900-2016 range (about 4% of all the car listings) and most of them occur in 2017 and 2018. To determine the minimum acceptable registration year, we go a step further by considering the listings from 1900-1999.

In [39]:
autos[(autos['registration_year']>1899)
      &(autos['registration_year']<2000)]['registration_year'].value_counts().sort_index()

1910       5
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       3
1951       2
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       4
1959       6
1960      23
1961       6
1962       4
1963       8
1964      12
1965      17
1966      22
1967      26
1968      26
        ... 
1970      38
1971      26
1972      33
1973      23
1974      24
1975      18
1976      21
1977      22
1978      44
1979      34
1980      85
1981      28
1982      41
1983      51
1984      51
1985      95
1986      72
1987      72
1988     135
1989     174
1990     347
1991     339
1992     370
1993     425
1994     629
1995    1227
1996    1373
1997    1951
1998    2363
1999    2897
Name: registration_year, Length: 61, dtype: int64

The listings here start from 1910 and though the number of listings increases from here on average to 1999, 1910 represents an acceptable starting point for the registration_year.

Therefore we can deduce that the lowest and highest acceptable values for the registration_year column are 1910 and 2016 respectively.

Now we will remove the values outside those lower and upper bounds for registration_year 

In [40]:
autos.shape

(48565, 18)

In [41]:
autos = autos[autos['registration_year'].between(1910,2016)]
autos.shape

(46681, 18)

So 1884 car listings were removed. Now we will also remove values which have a registration_month of 0. As was stated earlier there were 4480 rows with a registration_month of 0, however, some of the listings we removed from outside our acceptable registration_year range also had a registration month of 0 so now we will find out how many car listings with a registration month of 0 were within our acceptable registration_year range.

In [42]:
autos = autos[autos['registration_month']!=0]
autos.shape

(42655, 18)

About 4026 listings were removed, which represents the cars that were within our our registration_year range but did not match our registration_month criteria. Now we will calculate the distribution of the remaining registration_month values using Series.value_counts(normalize=True).

In [43]:
autos['registration_year'].value_counts(normalize=True).head(10)

2005    0.063252
1999    0.061282
2000    0.060814
2004    0.060649
2006    0.060392
2003    0.059501
2001    0.056640
2002    0.054460
2007    0.051436
2008    0.050123
Name: registration_year, dtype: float64

It seems that the majority of vehicles were first registered within the past 20 years.

## Exploring Price by Brand

The brand of a car could play a key factor on its sales figures as people may tend to buy brands they are accustomed to or that they perceive as desirable. The brand of a car may consequently affect the supply and demand of said car, hence it makes sense to delve deeper into the car brands. First we will look at the distribution of car sales by brand.

In [44]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.208229
bmw               0.112015
opel              0.103903
mercedes_benz     0.099730
audi              0.088032
ford              0.068738
renault           0.046466
peugeot           0.030008
fiat              0.025437
seat              0.018357
skoda             0.017044
nissan            0.015473
mazda             0.014934
smart             0.014441
citroen           0.013808
toyota            0.013293
hyundai           0.010245
mini              0.009354
sonstige_autos    0.009331
volvo             0.009284
mitsubishi        0.008018
honda             0.007830
kia               0.007479
alfa_romeo        0.006611
porsche           0.006400
suzuki            0.006002
chevrolet         0.005744
chrysler          0.003470
dacia             0.002766
jeep              0.002368
daihatsu          0.002344
land_rover        0.002180
subaru            0.002016
saab              0.001711
jaguar            0.001571
daewoo            0.001500
rover             0.001313
l

In terms of popularity, the car brands are dominated by German companies, who hold the top 4 positions, making up about 50% of the total listings. Volkswagen is the clear favourite making up about 20.8% of sales about double that of either of the next 2 competitors.

There are many brands without a notable percentage of listings, so we will limit our analysis to brands representing more than 5% of total listings.

In [67]:
brand_counts = autos['brand'].value_counts(normalize=True)
popular_brands = brand_counts[brand_counts > 0.05].index
popular_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')

So there are 6 brands which fit this criteria. Let's filter out our dataframe to only consist of these popular brands and store the filtered result in a new dataframe.

In [64]:
popular_autos = autos[autos['brand'].isin(popular_brands)]

Now we will look at the mean prices per brand.

In [68]:
mean_price_by_brand = (
    popular_autos.groupby('brand')['price_dollars']
    .mean().sort_index()
)
mean_price_by_brand

brand
audi             9776.165113
bmw              8652.106111
ford             3957.076739
mercedes_benz    8882.735778
opel             3167.165162
volkswagen       5752.935150
Name: price_dollars, dtype: float64

Out of the main brands, Ford and Opel are less expensive, while Audi, BMW and Mercedes Benz are more expensive. The top brand Volkswagen is in the middle price range, which may explain its popularity as it sits in between the lower range cars and higher range cars boasting price accessibility while maintaining a certain level of prestige.

## Investigating Mileage

Lastly, we will check to see if there is a connection with the average mileage of the cars from the main brands with their mean price.

In [72]:
popular_autos.groupby('brand').agg({'price_dollars': 'mean','odometer_km': 'mean'})

,odometer_km,price_dollars
brand,,
audi,128593.874834,9776.165113
bmw,132489.535370,8652.106111
ford,124009.208731,3957.076739
mercedes_benz,130927.362482,8882.735778
opel,128752.256318,3167.165162
volkswagen,128183.967575,5752.935150


It can be deduced from this that the range of car mileages (odometer_km) does not vary as much as the prices do by brand. However, there is a slight trend to the less expensive cars having lower mileage, with the more expensive vehicles having higher mileage.